In [1]:
# ignore warnings

import warnings
warnings.filterwarnings("ignore")

Installing Dependencies

In [2]:
!pip install accelerate==0.26.1
!pip install bitsandbytes==0.42.0
!pip install datasets==2.16.1
!pip install transformers==4.37.2
!pip install trl==0.7.11
!pip install ninja==1.11.1.1
!pip install pyarrow==15.0.0
!pip install einops==0.7.0
!pip install pandas==1.5.3
!pip install torch==2.1.0
!pip install rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

Load dataset

In [3]:
from datasets import load_dataset
multi_news = load_dataset("multi_news", split="train")

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

View Data

In [4]:
multi_news.to_pandas()

,document,summary
0,"National Archives \n \n Yes, it’s that time ag...",– The unemployment rate dropped to 8.2% last m...
1,LOS ANGELES (AP) — In her first interview sinc...,"– Shelly Sterling plans ""eventually"" to divorc..."
2,"GAITHERSBURG, Md. (AP) — A small, private jet ...",– A twin-engine Embraer jet that the FAA descr...
3,Tucker Carlson Exposes His Own Sexism on Twitt...,– Tucker Carlson is in deep doodoo with conser...
4,A man accused of removing another man's testic...,– What are the three most horrifying words in ...
...,...,...
44967,"More than 670,000 copies of the Pearls’ self-p...",– The deaths of three children have been linke...
44968,Seeking out cost-conscious consumers who have ...,"– Apple is hoping its new, cheaper iPhone can ..."
44969,Click to email this to a friend (Opens in new ...,"– January Jones, who plays the beleaguered wif..."
44970,"BARRINGTON, R.I. (AP) — Women clad in yoga pan...",– A Rhode Island man who penned a letter to th...


Train test split (80:20 ratio respectively)

In [5]:
multi_news = multi_news.train_test_split(test_size=0.2)

Tokenizing Training and testing sets

In [6]:
! pip install transformers

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [8]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
tokenized_multi_news = multi_news.map(preprocess_function, batched=True)

Map:   0%|          | 0/35977 [00:00<?, ? examples/s]

Map:   0%|          | 0/8995 [00:00<?, ? examples/s]

Model Loading

In [10]:
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='t5-small')
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Defining Hyperparameters

In [17]:
training_args = Seq2SeqTrainingArguments(
output_dir="./results",
evaluation_strategy="epoch",
learning_rate=2e-5,
per_device_train_batch_size=10,
per_device_eval_batch_size=10,
weight_decay=0.01,
save_total_limit=3,
num_train_epochs=3,
fp16=True,
)


Initializing Trainer

In [18]:
trainer = Seq2SeqTrainer(
model=model,
args=training_args,
train_dataset=tokenized_multi_news["train"],
eval_dataset=tokenized_multi_news["test"],
tokenizer=tokenizer,
data_collator=data_collator,
)


Train the Text Summarization Model

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.929000,2.719164


Evaluate model on a single document

In [ ]:
document = "National Archives Yes, it's that time again, folks. It's the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month. Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February. The unemployment rate is expected to hold steady at 8.3%. Here at MarketBeat HQ, we’ll be offering color commentary before and after the data crosses the wires. Feel free to weigh-in yourself, via the comments section. And while you're here, why don't you sign up to follow us on Twitter. Enjoy the show. ||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy may not be growing fast enough to sustain robust job growth. The unemployment rate dipped, but mostly because more Americans stopped looking for work. The Labor Department says the economy added 120,000 jobs in March, down from more than 200,000 in each of the previous three months. The unemployment rate fell to 8.2 percent, the lowest since January 2009. The rate dropped because fewer people searched for jobs. The official unemployment tally only includes those seeking work. The economy has added 858,000 jobs since December _ the best four months of hiring in two years. But Federal Reserve Chairman Ben Bernanke has cautioned that the current hiring pace is unlikely to continue without more consumer spending."
human_summary = """– The unemployment rate dropped to 8.2% last month, but the economy only added 120,000 jobs, when 203,000 new jobs had been predicted, according to today's jobs report. Reaction on the Wall Street Journal's MarketBeat Blog was swift: "Woah!!! Bad number." The unemployment rate, however, is better news; it had been expected to hold steady at 8.3%. But the AP notes that the dip is mostly due to more Americans giving up on seeking employment."""

In [ ]:
def predict_summary(document):
  device = model.device
  tokenized = tokenizer([document], truncation =True, padding ='longest',return_tensors='pt')
  tokenized = {k: v.to(device) for k, v in tokenized.items()}
  tokenized_result = model.generate(**tokenized, max_length=128)
  tokenized_result = tokenized_result.to('cpu')
  predicted_summary = tokenizer.decode(tokenized_result[0])
  return predicted_summary

In [ ]:
human_summary

In [ ]:
predicted_summary = predict_summary(document)
predicted_summary

Saving Model to HuggingFace

In [ ]:
from huggingface_hub import interpreter_login

interpreter_login() #Have to enter the huggingface hf token

In [ ]:
model.push_to_hub("Fine_Tune_T5")
tokenizer.push_to_hub("Fine_Tune_T5")

Evaluate using Rouge Scores

In [ ]:
from rouge import Rouge

def get_rouge_scores(actual_summary, predicted_summary):
    rouge = Rouge()
    scores = rouge.get_scores(predicted_summary, actual_summary)
    return [scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']]

In [ ]:
# import validation dataset

import pandas as pd
val_data = pd.read_excel('/content/Validation_Data.xlsx')
val_data

,document,summary
0,"Story highlights Officers circled the man, who...","– Angry relatives are calling it an ""execution..."
1,Ebola patient in critical condition in Texas h...,– The Ebola patient who's in critical conditio...
2,Skip in Skip x Embed x Share CLOSE President O...,"– Hold your fire, troops: President Obama made..."
3,William Aitcheson stepped down after revealing...,– A Catholic priest in Virginia has stepped do...
4,Taylor Swift's epic treadmill fall in Apple Mu...,"– On April 1, Taylor Swift shared an ad for Ap..."
...,...,...
95,Locked-In Man \n \n Invisibilia's Lulu Miller ...,– Martin Pistorius was a kid so into electroni...
96,Iraqi security forces celebrate as they pose w...,– Iraqi special forces swept into Fallujah on ...
97,Treasury Secretary Timothy F. Geithner said on...,– Tim Geithner is about to enact some “extraor...
98,"Votes for Mickey Mouse, football coach Nick Sa...",– On Wednesday Vice President Mike Pence will ...


In [ ]:
# rouge score of validation data

from tqdm import tqdm

rouge1_scores = []
rouge2_scores = []
rougel_scores = []

pred_summary_list = []

for i in tqdm(range(len(val_data))):

  doc = val_data.loc[i]['document']
  pred_summary = predict_summary(document)
  human_summary = val_data.loc[i]['summary']

  score = get_rouge_scores(human_summary, pred_summary)

  rouge1_scores.append(score[0])
  rouge2_scores.append(score[1])
  rougel_scores.append(score[2])

  pred_summary_list.append(pred_summary)

val_data["pred_summary"] = pred_summary_list

val_data['rouge1'] = rouge1_scores
val_data['rouge2'] = rouge2_scores
val_data['rougel'] = rougel_scores

val_data

100%|██████████| 100/100 [01:59<00:00,  1.20s/it]


,document,summary,rouge1,rouge2,rougel,pred_summary
0,"Story highlights Officers circled the man, who...","– Angry relatives are calling it an ""execution...",0.104265,0.013289,0.094787,"<pad><extra_id_0>, and the New York Times repo..."
1,Ebola patient in critical condition in Texas h...,– The Ebola patient who's in critical conditio...,0.117117,0.006309,0.117117,"<pad><extra_id_0>, and the New York Times repo..."
2,Skip in Skip x Embed x Share CLOSE President O...,"– Hold your fire, troops: President Obama made...",0.086486,0.000000,0.086486,"<pad><extra_id_0>, and the New York Times repo..."
3,William Aitcheson stepped down after revealing...,– A Catholic priest in Virginia has stepped do...,0.119266,0.020270,0.110092,"<pad><extra_id_0>, and the New York Times repo..."
4,Taylor Swift's epic treadmill fall in Apple Mu...,"– On April 1, Taylor Swift shared an ad for Ap...",0.149425,0.018265,0.126437,"<pad><extra_id_0>, and the New York Times repo..."
...,...,...,...,...,...,...
95,Locked-In Man \n \n Invisibilia's Lulu Miller ...,– Martin Pistorius was a kid so into electroni...,0.072131,0.004505,0.072131,"<pad><extra_id_0>, and the New York Times repo..."
96,Iraqi security forces celebrate as they pose w...,– Iraqi special forces swept into Fallujah on ...,0.096491,0.006452,0.096491,"<pad><extra_id_0>, and the New York Times repo..."
97,Treasury Secretary Timothy F. Geithner said on...,– Tim Geithner is about to enact some “extraor...,0.106509,0.000000,0.106509,"<pad><extra_id_0>, and the New York Times repo..."
98,"Votes for Mickey Mouse, football coach Nick Sa...",– On Wednesday Vice President Mike Pence will ...,0.106557,0.012579,0.090164,"<pad><extra_id_0>, and the New York Times repo..."


In [ ]:
# average rouge 1
val_data['rouge1'].mean()

0.11723885568649839

In [ ]:
# average rouge 2
val_data['rouge2'].mean()

0.012346104257379096

In [ ]:
# average rouge l
val_data['rougel'].mean()

0.1114219289972185